In [6]:
# Dependencies and Setup
import pandas as pd
import csv


# File to Load (Remember to Change These)
schools = "Resources/schools_complete.csv"
students= "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(schools)
student_data = pd.read_csv(students)

# Combine the data into a single dataset.  
school_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_complete


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [7]:
#Define new variables for totals

total_schools = len(school_complete["school_name"].unique())
total_budget = school_data["budget"].sum()
avg_math = round(school_complete["math_score"].mean(),2)
avg_reading = round(school_complete["reading_score"].mean(),2)


pass_math = len(school_complete.loc[(school_complete["math_score"] >= 70)])
total_students = len(school_complete["student_name"])
percent_math = (pass_math/total_students)

pass_read = len(school_complete.loc[(school_complete["reading_score"] >= 70)])
percent_read = (pass_read/total_students)

pass_read_math = len(school_complete.loc[(school_complete["math_score"] >= 70) & (school_complete["reading_score"] >=70)])
percent_bothpass = (pass_read_math/total_students)


#Create new dataframe to hold the new variables

district_summary = pd.DataFrame({"Total Number of Schools": [total_schools],
                                 "Total Number of Students": total_students,
                                "Total Budget":total_budget,
                                "Average Math Score": avg_math,
                                "Average Reading Score": avg_reading,
                                "Percentage of Students with a Passing Math Score": percent_math,
                                "Percentage of Students with a Passing Reading Score": percent_read,
                                "Percentage of Overall Passing Students ": percent_bothpass,
                                })

#Format dataframe so it looks 'cleaner'
district_summary.style.format({"Total Budget": "${:,}",
                               "Average Math Score": "{:.1f}",
                                "Average Reading Score": "{:.1f}",
                                "Percentage of Students with a Passing Math Score": "{:.1%}",
                                "Percentage of Students with a Passing Reading Score": "{:.1%}",
                                "Percentage of Overall Passing Students ": "{:.1%}"})

,Total Number of Schools,Total Number of Students,Total Budget,Average Math Score,Average Reading Score,Percentage of Students with a Passing Math Score,Percentage of Students with a Passing Reading Score,Percentage of Overall Passing Students
0,15,39170,"$24,649,428",79.0,81.9,75.0%,85.8%,65.2%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [8]:
#Only include columns that are necessary for analysis
school_reduced = school_complete[["school_name","type",
                                  "student_name", 
                                  "size", "budget", 
                                  "reading_score", "math_score"]]


#Set index to the names of the high schools in the district 
school_type = school_data.set_index('school_name')['type']

#For each variable, group by the school name

students_school = school_reduced.groupby('school_name')['student_name'].count()

school_budget = school_data.set_index('school_name')['budget']

school_student_budget = (school_budget / students_school)

school_math = school_reduced.groupby('school_name')['math_score'].mean()

schoolreading = school_reduced.groupby('school_name')['reading_score'].mean()

mathpass = (school_complete[school_complete["math_score"] >= 70].groupby('school_name')['student_name'].count())/students_school

readpass = (school_complete[school_complete["reading_score"] >= 70].groupby('school_name')['student_name'].count())/students_school

bothpass = (school_complete[(school_complete["math_score"] >= 70) & (school_complete["reading_score"] >=70)].groupby('school_name')['student_name'].count())/students_school


#Create new dataframe to hold newly created variables

schoolsummarytable = pd.DataFrame ({
                                   "School Type": school_type,
                                   "Total Students": students_school,
                                   "Total School Budget": school_budget,
                                   "Per School Budget": school_student_budget,
                                   "Average Math Score ": school_math,
                                   "Average Reading Score": schoolreading,
                                   "% Passing Math": mathpass,
                                   "% Passing Reading": readpass,
                                   "% Overall Passing": bothpass})

schoolsummarytable.style.format({"Total Students": "{:,}",
                                "Total School Budget": "${:,}",
                                "Per School Budget": "${:,}",
                                "Average Math Score ": "{:.1f}",
                                "Average Reading Score": "{:.1f}",
                                "% Passing Math": "{:.1%}",
                                "% Passing Reading": "{:.1%}",
                                "% Overall Passing": "{:.1%}"})




,School Type,Total Students,Total School Budget,Per School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928",$628.0,77.0,81.0,66.7%,81.9%,54.6%
Cabrera High School,Charter,"1,858","$1,081,356",$582.0,83.1,84.0,94.1%,97.0%,91.3%
Figueroa High School,District,"2,949","$1,884,411",$639.0,76.7,81.2,66.0%,80.7%,53.2%
Ford High School,District,"2,739","$1,763,916",$644.0,77.1,80.7,68.3%,79.3%,54.3%
Griffin High School,Charter,"1,468","$917,500",$625.0,83.4,83.8,93.4%,97.1%,90.6%
Hernandez High School,District,"4,635","$3,022,020",$652.0,77.3,80.9,66.8%,80.9%,53.5%
Holden High School,Charter,427,"$248,087",$581.0,83.8,83.8,92.5%,96.3%,89.2%
Huang High School,District,"2,917","$1,910,635",$655.0,76.6,81.2,65.7%,81.3%,53.5%
Johnson High School,District,"4,761","$3,094,650",$650.0,77.1,81.0,66.1%,81.2%,53.5%
Pena High School,Charter,962,"$585,858",$609.0,83.8,84.0,94.6%,95.9%,90.5%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [9]:
#Sort % Overall Passing for top performing schools

top_perform = schoolsummarytable.sort_values("% Overall Passing", ascending=False).head(5)

top_perform.style.format({"Total Students": "{:,}",
                                "Total School Budget": "${:,}",
                                "Per School Budget": "${:,}",
                                "Average Math Score ": "{:.1f}",
                                "Average Reading Score": "{:.1f}",
                                "% Passing Math": "{:.1%}",
                                "% Passing Reading": "{:.1%}",
                                "% Overall Passing": "{:.1%}"})


,School Type,Total Students,Total School Budget,Per School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356",$582.0,83.1,84.0,94.1%,97.0%,91.3%
Thomas High School,Charter,"1,635","$1,043,130",$638.0,83.4,83.8,93.3%,97.3%,90.9%
Griffin High School,Charter,"1,468","$917,500",$625.0,83.4,83.8,93.4%,97.1%,90.6%
Wilson High School,Charter,"2,283","$1,319,574",$578.0,83.3,84.0,93.9%,96.5%,90.6%
Pena High School,Charter,962,"$585,858",$609.0,83.8,84.0,94.6%,95.9%,90.5%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [10]:
#Use same sort function from above minus 'ascending = False' clause

low_perform = schoolsummarytable.sort_values("% Overall Passing").head(5)

low_perform.style.format({"Total Students": "{:,}",
                                "Total School Budget": "${:,}",
                                "Per School Budget": "${:,}",
                                "Average Math Score ": "{:.1f}",
                                "Average Reading Score": "{:.1f}",
                                "% Passing Math": "{:.1%}",
                                "% Passing Reading": "{:.1%}",
                                "% Overall Passing": "{:.1%}"})


,School Type,Total Students,Total School Budget,Per School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363",$637.0,76.8,80.7,66.4%,80.2%,53.0%
Figueroa High School,District,"2,949","$1,884,411",$639.0,76.7,81.2,66.0%,80.7%,53.2%
Huang High School,District,"2,917","$1,910,635",$655.0,76.6,81.2,65.7%,81.3%,53.5%
Hernandez High School,District,"4,635","$3,022,020",$652.0,77.3,80.9,66.8%,80.9%,53.5%
Johnson High School,District,"4,761","$3,094,650",$650.0,77.1,81.0,66.1%,81.2%,53.5%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [11]:
#Select each grade from student data and group by school name

ninth = student_data[student_data['grade'] == "9th"].groupby('school_name')['math_score'].mean()
tenth = student_data[student_data['grade'] == "10th"].groupby('school_name')['math_score'].mean()
eleventh = student_data[student_data['grade'] == "11th"].groupby('school_name')['math_score'].mean()
twelfth = student_data[student_data['grade'] == "12th"].groupby('school_name')['math_score'].mean()

mathscoregrade = pd.DataFrame({"9th Grade": ninth,
                              "10th Grade": tenth,
                              "11th Grade": eleventh,
                              "12th Grade": twelfth})

mathscoregrade.style.format({"9th Grade": "{:.1f}",
                            "10th Grade": "{:.1f}",
                            "11th Grade": "{:.1f}",
                            "12th Grade": "{:.1f}" })

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [12]:
nine = student_data[student_data['grade'] == "9th"].groupby('school_name')['reading_score'].mean()
ten = student_data[student_data['grade'] == "10th"].groupby('school_name')['reading_score'].mean()
eleven = student_data[student_data['grade'] == "11th"].groupby('school_name')['reading_score'].mean()
twelve = student_data[student_data['grade'] == "12th"].groupby('school_name')['reading_score'].mean()

readscoregrade = pd.DataFrame({"9th Grade": nine,
                              "10th Grade": ten,
                              "11th Grade": eleven,
                              "12th Grade": twelve})

readscoregrade.style.format({"9th Grade": "{:.1f}",
                            "10th Grade": "{:.1f}",
                            "11th Grade": "{:.1f}",
                            "12th Grade": "{:.1f}" })

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [23]:
# Create bins to hold the different ranges
bins = [0,583.999,629.999,644.999,674.999]
binlabel = ["<$584", "$584-629", "$630-644", "$645-675"]
schoolsummarytable ["Spending Ranges (per Student)"] = pd.cut(school_student_budget, bins, labels = binlabel, include_lowest= True)

spendrange = schoolsummarytable

#Create new variables that are grouped by the new variable 
binmath = spendrange.groupby(["Spending Ranges (per Student)"]).mean()["Average Math Score "]

binread = spendrange.groupby(["Spending Ranges (per Student)"]).mean()["Average Reading Score"]

mathpercentbin = spendrange.groupby(["Spending Ranges (per Student)"]).mean()["% Passing Math"]

readpercentbin = spendrange.groupby(["Spending Ranges (per Student)"]).mean()["% Passing Reading"]

overallbin = spendrange.groupby(["Spending Ranges (per Student)"]).mean()["% Overall Passing"]

update = pd.DataFrame({"Average Math Score ": round(binmath,2),
                      "Average Reading Score": round(binread,2),
                      "% Passing Math": round(mathpercentbin,2),
                      "% Passing Reading": round(readpercentbin * 100,2),
                      "% Overall Passing": round(overallbin * 100,2)})

update.head()




,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (per Student),,,,,
<$584,83.46,83.93,0.93,96.61,90.37
$584-629,81.90,83.16,0.87,92.72,81.42
$630-644,78.52,81.62,0.73,84.39,62.86
$645-675,77.00,81.03,0.66,81.13,53.53



## Scores by School Size

* Perform the same operations as above, based on school size.

In [25]:
bins2 = [0,999.999,1999.999,4999.999]
binlabel2 = ["<1000", "1000-2000", "2000-5000"]

schoolsummarytable["School Size"] = pd.cut(schoolsummarytable["Total Students"], bins2, labels = binlabel2, include_lowest= True)

schoolsize = schoolsummarytable

binmath1 = schoolsize.groupby(["School Size"]).mean()["Average Math Score "]

binread1 = schoolsize.groupby(["School Size"]).mean()["Average Reading Score"]

mathpercentbin1 = schoolsize.groupby(["School Size"]).mean()["% Passing Math"]

readpercentbin1 = schoolsize.groupby(["School Size"]).mean()["% Passing Reading"]

overallbin1 = schoolsize.groupby(["School Size"]).mean()["% Overall Passing"]


schoolbin = pd.DataFrame({"Average Math Score": round(binmath1,2),
                         "Average Reading Score": round(binread1,2),
                         "% Passing Math": round(mathpercentbin1 * 100 ,2),
                         "% Passing Reading": round(readpercentbin1 * 100, 2),
                         "% Overall Passing": round(overallbin1 * 100,2)})

schoolbin.head()





,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
<1000,83.82,83.93,93.55,96.10,89.88
1000-2000,83.37,83.86,93.60,96.79,90.62
2000-5000,77.75,81.34,69.96,82.77,58.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [27]:
spendrangecuts = spendrange.drop(["Total Students", "Total School Budget", "Per School Budget","Spending Ranges (per Student)",
                                 "Spending Ranges (per Student)", "School Size"], axis = 1)
updated_type = spendrangecuts

typemath  = updated_type.groupby(["School Type"]).mean()["Average Math Score "]

typeread = updated_type.groupby(["School Type"]).mean()["Average Reading Score"]

mathpercenttype = updated_type.groupby(["School Type"]).mean()["% Passing Math"]

readpercenttype = updated_type.groupby(["School Type"]).mean()["% Passing Reading"]

overalltype = updated_type.groupby(["School Type"]).mean()["% Overall Passing"]


newtype = pd.DataFrame ({"Average Math Score": round(typemath,2),
                         "Average Reading Score": round(typeread,2),
                         "% Passing Math": round(mathpercenttype * 100 ,2),
                         "% Passing Reading": round(readpercenttype * 100, 2),
                         "% Overall Passing": round(overalltype * 100,2)})

newtype.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
